## **Overview and Purpose**

The Purpose of the notebook is to conduct a data exploration exercise on the IMDB Top 5000 movies that would answer the below questions:
* What do the Plot Key words, Movie Title and Genres of teh top movies indicate ?
* What is the trend for the Gross Revenue and Budgetof the movies in nominal terms as well as inflation adjusted terms ?
* Who are the top actors, directors and movies across decades ?

In [1]:
library(ggplot2) # Data visualization
library(readr) # CSV file I/O, e.g. the read_csv function
list.files("../input")
library(tm)
library(SnowballC)
library(wordcloud)
library(RColorBrewer) 
library(dplyr)
library(tidyr)
library(reshape2)
library(gganimate)
options(warn=-1)
movie.data <- read.csv("../input/movie_metadata.csv",header = TRUE)

## **Word Clouds on Plot Key words, Movie Title and Genres**

In [39]:
options(repr.plot.width=8, repr.plot.height=8)
#Word-cloud for Plot Keywords
corpus <- Corpus(VectorSource(gsub("[^a-zA-Z]+", " ", movie.data$plot_keywords)))
corpus <- tm_map(corpus, PlainTextDocument)
corpus <- tm_map(corpus, removeWords , stopwords('english'))
corpus <- tm_map(corpus, removeWords , c('word','actor','cast','title','based'))
corpus <- Corpus(VectorSource(corpus))
pal2 <- brewer.pal(8,"Dark2")
wordcloud(corpus,scale=c(8,1), max.words=500, random.order=FALSE, rot.per=.15, colors=pal2 )


In [3]:
#Word-cloud for Movie Titles
corpus <- Corpus(VectorSource(gsub("[^a-zA-Z]+", " ", movie.data$movie_title)))
corpus <- tm_map(corpus, PlainTextDocument)
corpus <- tm_map(corpus, removePunctuation)
corpus <- tm_map(corpus, removeWords , stopwords('english'))
corpus <- tm_map(corpus, removeWords , c("the","The","THE"))
corpus <- Corpus(VectorSource(corpus))
pal2 <- brewer.pal(8,"Dark2")
wordcloud(corpus,scale=c(8,1), max.words=100, random.order=FALSE, rot.per=.20, colors=pal2)

In [4]:
#Word-cloud for Movie Genres
corpus <- Corpus(VectorSource(gsub("[^a-zA-Z]+", " ", movie.data$genres)))
corpus <- tm_map(corpus, PlainTextDocument)
corpus <- tm_map(corpus, removePunctuation)
corpus <- tm_map(corpus, removeWords , stopwords('english'))
corpus <- tm_map(corpus, removeWords , c("the","The","THE","film"))
corpus <- Corpus(VectorSource(corpus))
pal2 <- brewer.pal(8,"Dark2")
wordcloud(corpus,scale=c(8,1), max.words=100, random.order=FALSE, rot.per=.15, colors=pal2)

**Mining Insights based on the Word clouds:**
* The Plot keywords paints a dark and gruesome picture of the world. In fact a sentimental analysis on the plot keywords returns a negative view of the world composed of death, nudity and love.
* For the Movie Title word cloud, there are surprisingly lot of movie titles with the word "Movie" & "Story" in them.
* Also in the Movie Title word cloud, we see a dark world comprising of death and love. I wonder why are movies based on dark themes ?
* From the genre word cloud, it is clear that majority of the movies belong to Drama and Comedy Genre with the Romantic genre coming in a close second.
* If we compare the plot keywords with the genre word cloud, it is hard to see why the positivity of the comedy movies is not reflected in the Plot keywords word cloud


## ** Understanding revenue and budget trends across 100 years**

We plot the budget and revenue across decades, first in nominal terms and then adjusting according to the inflation index to visualize the trends across time

Note: The Next bunch of code is to incorporate the CPI Index into the dataset

In [5]:
#Creating the Inflation Index Data Frame from the actual Labour Department CPI Index.
## Note: If someone knows how to load the data directly into a Kaggle kernel from outside, then please help!
x <- c(1913,1914,1915,       1916,       1917,       1918,       1919,       1920,       1921,       1922,       1923,       1924,       1925,       1926,       1927,       1928,       1929,       1930,       1931,       1932,       1933,       1934,       1935,       1936,       1937,       1938,       1939,       1940,       1941,       1942,       1943,       1944,       1945,       1946,       1947,       1948,       1949,       1950,       1951,       1952,       1953,       1954,       1955,       1956,       1957,       1958,       1959,       1960,       1961,       1962,       1963,       1964,       1965,       1966,       1967,       1968,       1969,       1970,       1971,       1972,       1973,       1974,       1975,       1976,       1977,       1978,       1979,       1980,       1981,       1982,       1983,       1984,       1985,       1986,       1987,       1988,       1989,       1990,       1991,       1992,       1993,       1994,       1995,       1996,       1997,       1998,       1999,       2000,       2001,       2002,       2003,       2004,       2005,       2006,       2007,       2008,       2009,       2010,       2011,       2012,       2013,       2014,       2015,       2016 )
y = c(9.9,      10,      10.1,      10.9,      12.8,      15.1,      17.3,      20,      17.9,      16.8,      17.1,      17.1,      17.5,      17.7,      17.4,      17.1,      17.1,      16.7,      15.2,      13.7,      13,      13.4,      13.7,      13.9,      14.4,      14.1,     13.9,      14,      14.7,      16.3,      17.3,      17.6,      18,      19.5,      22.3,      24.1,      23.8,      24.1,      26,      26.5,      26.7,      26.9,      26.8,      27.2,      28.1,      28.9,      29.1,      29.6,      29.9,      30.2,      30.6,      31,      31.5,      32.4,      33.4,      34.8,      36.7,      38.8,      40.5,      41.8,      44.4,      49.3,      53.8,      56.9,      60.6,      65.2,      72.6,      82.4,      90.9,      96.5,      99.6,      103.9,      107.6,      109.6,      113.6,      118.3,      124,      130.7,      136.2,      140.3,      144.5,      148.2,      152.4,      156.9,      160.5,      163,      166.6,      172.2,      177.1,      179.9,      184,      188.9,      195.3,      201.6,      207.3,      215.303,      214.537,      218.056,      224.939,      229.594,      232.957,      236.736,      237.017,      240.007) 
inf.index <- data.frame("Year" = x,"CPI"= y)

## Creating a new temporary variable Movie Revenue
movie.revenue <- movie.data %>%
  filter(!is.na(gross)) %>%
  filter(!is.na(budget)) %>%
  filter(!is.na(title_year))

# Associating the Inflation Index with the Movie Revenue
inf <- function(year) {
   new.cpi <- inf.index[which(inf.index$Year==2016),2]
   old.cpi <- inf.index[which(inf.index$Year==year),2]
   return ((new.cpi-old.cpi)/old.cpi)
 }
inf.ind <- sapply(movie.revenue$title_year,inf)
movie.revenue$inf.index <- inf.ind

In [6]:
#Creating a new variable Decade for visualizing each decade
movie.revenue$decade <- "None"
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1910,1920),"1910-1920",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1920,1930),"1920-1930",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1930,1940),"1930-1940",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1940,1950),"1940-1950",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1950,1960),"1950-1960",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1960,1970),"1960-1970",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1970,1980),"1970-1980",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1980,1990),"1980-1990",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,1990,2000),"1990-2000",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,2000,2010),"2000-2010",movie.revenue$decade)
movie.revenue$decade <- ifelse(between(movie.revenue$title_year,2010,2020),"2010-2016",movie.revenue$decade)

#Count of movies for each decade in the database
movie.revenue %>%
  count(decade)

In [7]:
#Plotting Gross Revenue of the movie across decades
options(repr.plot.width=8, repr.plot.height=4)
movie.revenue %>%
  filter(gross < 25000000) %>%
  ggplot() +
  geom_boxplot(aes(decade,gross,fill = decade) ) +
  scale_y_continuous(name="Gross Revnue", labels = scales::comma)+
  scale_fill_brewer(palette="Set3")+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Decade")

# Plotting Inflation Adjusted gross revenue acorss decades
movie.revenue %>%
  filter(gross < 25000000) %>%
  mutate(gross.adj = gross * inf.index , budget.adj = budget * inf.index) %>% 
  ggplot() +
  geom_boxplot(aes(decade,gross.adj, fill = decade))+
  scale_y_continuous(name="Inflation Adjusted Gross Revnue", labels = scales::comma)+
  scale_fill_brewer(palette="Set3")+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Decade")

In [8]:
#Plotting Budget of the movie across decades
options(repr.plot.width=8, repr.plot.height=4)
movie.revenue %>%
  filter(budget < 25000000) %>%
  ggplot() +
  geom_boxplot(aes(decade,budget,fill = decade) ) +
  scale_y_continuous(name="Budget", labels = scales::comma)+
  scale_fill_brewer(palette="Set3")+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Decade")

# Plotting Inflation Adjusted Budget acorss decades
movie.revenue %>%
  filter(budget < 25000000) %>%
  mutate(gross.adj = gross * inf.index , budget.adj = budget * inf.index) %>% 
  ggplot() +
  geom_boxplot(aes(decade,budget.adj, fill = decade))+
  scale_y_continuous(name="Inflation Adjusted Budget", labels = scales::comma)+
  scale_fill_brewer(palette="Set3")+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Decade")

In [9]:
#Plotting the Outliers for Gross Revnue and Budget in each decade
options(repr.plot.width=8, repr.plot.height=4)
movie.revenue %>%
  mutate(gross.adj = gross * inf.index , budget.adj = budget * inf.index) %>% 
  ggplot() +
  geom_boxplot(aes(decade,gross, fill = decade))+
  scale_y_continuous(name="Gross Revenue", labels = scales::comma)+
  scale_fill_brewer(palette="Set3")+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Decade")

movie.revenue %>%
  mutate(gross.adj = gross * inf.index , budget.adj = budget * inf.index) %>% 
  ggplot() +
  geom_boxplot(aes(decade,budget, fill = decade))+
  scale_y_continuous(name="Budget", labels = scales::comma)+
  scale_fill_brewer(palette="Set3")+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Decade")

**Mining Insights based on Gross Revenue and Budget Trends:**
* The Gross revue by itself seems to be random across decades. However, when we plot the inflation adjusted gross revenue we clearly see that it is nose-diving since the 60's decade. 
* The Budget on which movies are being made also do not show any pattern until we plot the Inflation adjusted Budget, which too clearly indicates that on inflation adjusted basis the budget on which movies are being made in Hollywood are going down.
* These graphs are indicative of the average trends as, when we plot the outliers they seem to be increasing every decade. Thus masking the real trend in the industry.

## ** Identifying the top directors, actors and movies across 100 years based on Gross Revenue**

In [32]:
# Identifying the Top 5 Directors in Each Decade based on Gross Revenue:
options(repr.plot.width=8, repr.plot.height=6)

top_directors <- movie.revenue %>%
  group_by(decade) %>%
  select(director_name,gross,decade,movie_title) %>%
  arrange(desc(gross)) %>%
  top_n(n=5,wt=gross)

ggplot(top_directors , aes(director_name,decade,color = decade , size = gross , label = movie_title) ) +
  geom_point()+
  coord_flip()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Director")+
  ylab("Decade")+
  ggtitle("Top Directors based on movie's Gross Revenue for each decade")

# Identifying the Top 5 Directors in Each Decade based on IMDB Rank:
top_directors <- movie.revenue %>%
  group_by(decade) %>%
  select(director_name,imdb_score,decade,movie_title) %>%
  arrange(desc(imdb_score)) %>%
  top_n(n=5,wt=imdb_score)

ggplot(top_directors , aes(director_name,decade,color = decade , size = imdb_score , label = movie_title) ) +
  geom_point()+
  coord_flip()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Director")+
  ylab("Decade")+
  ggtitle("Top Directors based on movie's IMDB Rank for each decade")

In [33]:
# Identifying the Top 5 Actors in Each Decade based on Gross Revenue:
options(repr.plot.width=8, repr.plot.height=8)
actor_names <- movie.revenue %>%
  select(actor_1_name,actor_2_name,actor_3_name,movie_title,decade,gross,budget,imdb_score)

actor_names <- melt(actor_names , id.vars = c("movie_title","decade","gross","budget","imdb_score"))

actor_names <- separate(data = actor_names, col = movie_title , into = c("movie_title","extra"), sep = "Â")

top_actors <- actor_names %>%
  group_by(decade) %>%
  select(value,gross,decade,movie_title) %>%
  arrange(desc(gross)) %>%
  top_n(n=5,wt=gross)

ggplot(top_actors , aes(value,decade,color = decade , size = gross , label = movie_title) ) +
  geom_point()+
  coord_flip()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Actor")+
  ylab("Decade")+
  ggtitle("Top Actors based on movie's Gross Revenue for each decade")

#Identifying Top Actors based on IMDB Rank:
top_actors <- actor_names %>%
  group_by(decade) %>%
  select(value,imdb_score,decade,movie_title) %>%
  arrange(desc(imdb_score)) %>%
  top_n(n=5,wt=imdb_score)

ggplot(top_actors , aes(value,decade,color = decade , size = imdb_score , label = movie_title) ) +
  geom_point()+
  coord_flip()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Actor")+
  ylab("Decade")+
  ggtitle("Top Actors based on movie's IMDB Rank for each decade")

In [34]:
# Identifying the Top 5 Movie Titles in Each Decade based on Gross Revenue:
movie_names <- separate(data = movie.revenue, col = movie_title , into = c("movie_title","extra"), sep = "Â")

top_movies <- movie_names %>%
  group_by(decade) %>%
  select(movie_title,gross,decade) %>%
  arrange(desc(gross)) %>%
  top_n(n=5,wt=gross)

ggplot(top_movies , aes(movie_title,decade,color = decade , size = gross , label = movie_title) ) +
  geom_point()+
  coord_flip()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Movie Titles")+
  ylab("Decade")+
  ggtitle("Top Movies based on Gross Revenue for each decade")

# Identifying the Top 5 Movie Titles in Each Decade based on IMDB Rank:
top_movies <- movie_names %>%
  group_by(decade) %>%
  select(movie_title,imdb_score,decade) %>%
  arrange(desc(imdb_score)) %>%
  top_n(n=5,wt=imdb_score)

ggplot(top_movies , aes(movie_title,decade,color = decade , size = imdb_score , label = movie_title) ) +
  geom_point()+
  coord_flip()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  xlab("Movie Titles")+
  ylab("Decade")+
  ggtitle("Top Movies based on IMDB Rank for each decade")

**Mining Insights on Top Movies, Actors and Directors**
* Clearly there is a striking difference in the movies which top the chart based on Gross Revenue and the movies that top the chart based on IMDB Rank for each decade. 
* The actors and directors associated with the top movies that gross highly are different than the actors and directors that feature in top movies based on IMDB Rank
* Surprisingly, female actors are under represented on the top actors list in both lists by IMDB Score and Gross Revenue. Although their under representation in the top movies by IMDB Rank is striking
* Christopher Nolan, James Cameroon, George Lucas, Steven Spielsberg, Robert Zemicks and David Lean have directed top movies in more than 1 decade
* Morgan Freeman is the only actor who seems to have acted in top movies by IMDB Rank acorss decade.
* Harrison Ford, Ian Macdiarmid, and Kenny Baker are the actors who have actop in top grossing movies across decades.